In [1]:
from init import *

```python
def token_create( issuer: int, maximum_supply: int, symbol: int) -> None
def token_issue( to: int, quantity: int, symbol: int, memo: Union[str, bytes]) -> None
def token_transfer( from_: int, to: int, quantity: int, symbol: int, memo: Union[str, bytes]) -> None
```


In [2]:
code = r'''
import struct
import chain

def apply(receiver, first_receiver, action):
    if not receiver == first_receiver:
        return
    if action == name('create'):
#        token_create( uint64_t issuer, int64_t maximum_supply, uint64_t sym)
        data = chain.read_action_data()
        issuer, maximum_supply, symbol = struct.unpack('QQQ', data)
        chain.token_create(issuer, maximum_supply, symbol)
    elif action == name('issue'):
# void token_issue( uint64_t to, int64_t quantity, uint64_t sym, const char* memo, size_t size2 );
        data = chain.read_action_data()
        to, quantity, symbol = struct.unpack('QQQ', data[:24])
        chain.token_issue(to, quantity, symbol, data[25:])
    elif action == name('transfer'):
# void token_transfer( uint64_t from, uint64_t to, int64_t quantity, uint64_t sym, const char* memo, size_t size2);
        data = chain.read_action_data()
        _from, to, quantity, symbol = struct.unpack('QQQQ', data[:32])
        chain.token_transfer(_from, to, quantity, symbol, data[33:])

'''
code = uuosapi.mp_compile(python_contract, code)
r = uuosapi.deploy_python_contract(python_contract, code, b'', deploy_type=1)
print('done!')

done!


In [3]:
abi = '''
{
    "version": "eosio::abi/1.1",
    "structs": [
        {
            "name": "account",
            "base": "",
            "fields": [
                {
                    "name": "balance",
                    "type": "asset"
                }
            ]
        },
        {
            "name": "close",
            "base": "",
            "fields": [
                {
                    "name": "owner",
                    "type": "name"
                },
                {
                    "name": "symbol",
                    "type": "symbol"
                }
            ]
        },
        {
            "name": "create",
            "base": "",
            "fields": [
                {
                    "name": "issuer",
                    "type": "name"
                },
                {
                    "name": "maximum_supply",
                    "type": "asset"
                }
            ]
        },
        {
            "name": "currency_stats",
            "base": "",
            "fields": [
                {
                    "name": "supply",
                    "type": "asset"
                },
                {
                    "name": "max_supply",
                    "type": "asset"
                },
                {
                    "name": "issuer",
                    "type": "name"
                }
            ]
        },
        {
            "name": "issue",
            "base": "",
            "fields": [
                {
                    "name": "to",
                    "type": "name"
                },
                {
                    "name": "quantity",
                    "type": "asset"
                },
                {
                    "name": "memo",
                    "type": "string"
                }
            ]
        },
        {
            "name": "open",
            "base": "",
            "fields": [
                {
                    "name": "owner",
                    "type": "name"
                },
                {
                    "name": "symbol",
                    "type": "symbol"
                },
                {
                    "name": "ram_payer",
                    "type": "name"
                }
            ]
        },
        {
            "name": "retire",
            "base": "",
            "fields": [
                {
                    "name": "quantity",
                    "type": "asset"
                },
                {
                    "name": "memo",
                    "type": "string"
                }
            ]
        },
        {
            "name": "transfer",
            "base": "",
            "fields": [
                {
                    "name": "from",
                    "type": "name"
                },
                {
                    "name": "to",
                    "type": "name"
                },
                {
                    "name": "quantity",
                    "type": "asset"
                },
                {
                    "name": "memo",
                    "type": "string"
                }
            ]
        }
    ],
    "types": [],
    "actions": [
        {
            "name": "close",
            "type": "close",
            "ricardian_contract": ""
        },
        {
            "name": "create",
            "type": "create",
            "ricardian_contract": ""
        },
        {
            "name": "issue",
            "type": "issue",
            "ricardian_contract": ""
        },
        {
            "name": "open",
            "type": "open",
            "ricardian_contract": ""
        },
        {
            "name": "retire",
            "type": "retire",
            "ricardian_contract": ""
        },
        {
            "name": "transfer",
            "type": "transfer",
            "ricardian_contract": ""
        }
    ],
    "tables": [
        {
            "name": "accounts",
            "type": "account",
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        },
        {
            "name": "stat",
            "type": "currency_stats",
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        },
        {
            "name": "log",
            "type": "string",
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        },
        {
            "name": "codecache",
            "type": "string",
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        },
        {
            "name": "codetable",
            "type": "string",
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        },
        {
            "name": "modules",
            "type": "string",
            "index_type": "i64",
            "key_names": [],
            "key_types": []
        }
    ],
    "ricardian_clauses": [],
    "variants": [],
    "abi_extensions": [],
    "error_messages": [],
    "abi_extensions": []
}
'''

In [4]:
r = uuosapi.deploy_abi(python_contract, abi)

In [5]:
symbol_name = 'QQQ'
args = {"issuer":python_contract,"maximum_supply":f"11000000000000.0000 {symbol_name}"}
r = uuosapi.push_action(python_contract, 'create', args, {python_contract:'active'})

In [6]:
args = {"to":python_contract,"quantity":f"1000000.0000 {symbol_name}", "memo":f"issue 1000000.0000 {symbol_name} to {python_contract}"}
r = uuosapi.push_action(python_contract, 'issue', args, {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

In [7]:
uuosapi.get_balance(python_contract, token_account=python_contract, token_name=symbol_name)

1000000.0

In [8]:
args = {'from':python_contract, 'to':'eosio', 'quantity':f'10.0000 {symbol_name}', 'memo':'hello,world'}
r = uuosapi.push_action(python_contract, 'transfer', args, {python_contract:'active'})

In [9]:
uuosapi.get_balance('eosio', token_account=python_contract, token_name=symbol_name)

10.0

In [10]:
uuosapi.get_balance(python_contract, token_account=python_contract, token_name=symbol_name)

999990.0